# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 1 2023 2:54PM,256429,10,9093845,"Snap Medical Industries, LLC",Released
1,Feb 1 2023 2:54PM,256428,10,PRF-51298,Bio-PRF,Released
2,Feb 1 2023 2:54PM,256428,10,PRF-51305,Bio-PRF,Released
3,Feb 1 2023 2:54PM,256428,10,PRF-51320,Bio-PRF,Released
4,Feb 1 2023 2:54PM,256428,10,PRF-51327,Bio-PRF,Released
5,Feb 1 2023 2:54PM,256428,10,PRF-51351,Bio-PRF,Released
6,Feb 1 2023 2:54PM,256428,10,PRF-51354,Bio-PRF,Released
7,Feb 1 2023 2:53PM,256426,10,MSP218111,"Methapharm, Inc.",Released
8,Feb 1 2023 2:53PM,256426,10,MSP218046,"Methapharm, Inc.",Released
9,Feb 1 2023 2:53PM,256426,10,MSP218100,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,256425,Released,1
43,256426,Released,9
44,256427,Released,1
45,256428,Released,6
46,256429,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256425,NaN,NaN,1.0
256426,NaN,NaN,9.0
256427,NaN,NaN,1.0
256428,NaN,NaN,6.0
256429,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256360,8.0,1.0,8.0
256362,4.0,11.0,0.0
256363,8.0,0.0,7.0
256370,0.0,3.0,35.0
256378,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256360,8,1,8
256362,4,11,0
256363,8,0,7
256370,0,3,35
256378,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256360,8,1,8
1,256362,4,11,0
2,256363,8,0,7
3,256370,0,3,35
4,256378,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256360,8,1,8
1,256362,4,11,
2,256363,8,,7
3,256370,,3,35
4,256378,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 1 2023 2:54PM,256429,10,"Snap Medical Industries, LLC"
1,Feb 1 2023 2:54PM,256428,10,Bio-PRF
7,Feb 1 2023 2:53PM,256426,10,"Methapharm, Inc."
16,Feb 1 2023 2:52PM,256427,19,"Emersa Waterbox, LLC"
17,Feb 1 2023 2:46PM,256424,10,ISDIN Corporation
37,Feb 1 2023 2:46PM,256425,10,"Methapharm, Inc."
38,Feb 1 2023 2:41PM,256423,10,Emerginnova
39,Feb 1 2023 2:41PM,256422,10,"Citieffe, Inc."
40,Feb 1 2023 2:33PM,256420,19,"AdvaGen Pharma, Ltd"
41,Feb 1 2023 2:29PM,256419,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 1 2023 2:54PM,256429,10,"Snap Medical Industries, LLC",,,1
1,Feb 1 2023 2:54PM,256428,10,Bio-PRF,,,6
2,Feb 1 2023 2:53PM,256426,10,"Methapharm, Inc.",,,9
3,Feb 1 2023 2:52PM,256427,19,"Emersa Waterbox, LLC",,,1
4,Feb 1 2023 2:46PM,256424,10,ISDIN Corporation,,,20
5,Feb 1 2023 2:46PM,256425,10,"Methapharm, Inc.",,,1
6,Feb 1 2023 2:41PM,256423,10,Emerginnova,,,1
7,Feb 1 2023 2:41PM,256422,10,"Citieffe, Inc.",,,1
8,Feb 1 2023 2:33PM,256420,19,"AdvaGen Pharma, Ltd",,,1
9,Feb 1 2023 2:29PM,256419,19,"AdvaGen Pharma, Ltd",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:54PM,256429,10,"Snap Medical Industries, LLC",1,,
1,Feb 1 2023 2:54PM,256428,10,Bio-PRF,6,,
2,Feb 1 2023 2:53PM,256426,10,"Methapharm, Inc.",9,,
3,Feb 1 2023 2:52PM,256427,19,"Emersa Waterbox, LLC",1,,
4,Feb 1 2023 2:46PM,256424,10,ISDIN Corporation,20,,
5,Feb 1 2023 2:46PM,256425,10,"Methapharm, Inc.",1,,
6,Feb 1 2023 2:41PM,256423,10,Emerginnova,1,,
7,Feb 1 2023 2:41PM,256422,10,"Citieffe, Inc.",1,,
8,Feb 1 2023 2:33PM,256420,19,"AdvaGen Pharma, Ltd",1,,
9,Feb 1 2023 2:29PM,256419,19,"AdvaGen Pharma, Ltd",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:54PM,256429,10,"Snap Medical Industries, LLC",1,,
1,Feb 1 2023 2:54PM,256428,10,Bio-PRF,6,,
2,Feb 1 2023 2:53PM,256426,10,"Methapharm, Inc.",9,,
3,Feb 1 2023 2:52PM,256427,19,"Emersa Waterbox, LLC",1,,
4,Feb 1 2023 2:46PM,256424,10,ISDIN Corporation,20,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:54PM,256429,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
1,Feb 1 2023 2:54PM,256428,10,Bio-PRF,6.0,NaN,NaN
2,Feb 1 2023 2:53PM,256426,10,"Methapharm, Inc.",9.0,NaN,NaN
3,Feb 1 2023 2:52PM,256427,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Feb 1 2023 2:46PM,256424,10,ISDIN Corporation,20.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 1 2023 2:54PM,256429,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
1,Feb 1 2023 2:54PM,256428,10,Bio-PRF,6.0,0.0,0.0
2,Feb 1 2023 2:53PM,256426,10,"Methapharm, Inc.",9.0,0.0,0.0
3,Feb 1 2023 2:52PM,256427,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Feb 1 2023 2:46PM,256424,10,ISDIN Corporation,20.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5128189,104.0,26.0,7.0
12,512815,1.0,1.0,0.0
15,512723,15.0,1.0,16.0
19,2307685,25.0,17.0,1.0
21,512783,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5128189,104.0,26.0,7.0
1,12,512815,1.0,1.0,0.0
2,15,512723,15.0,1.0,16.0
3,19,2307685,25.0,17.0,1.0
4,21,512783,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104.0,26.0,7.0
1,12,1.0,1.0,0.0
2,15,15.0,1.0,16.0
3,19,25.0,17.0,1.0
4,21,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,12,Released,1.0
2,15,Released,15.0
3,19,Released,25.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21
Status,,,,,
Completed,7.0,0.0,16.0,1.0,0.0
Executing,26.0,1.0,1.0,17.0,0.0
Released,104.0,1.0,15.0,25.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21
0,Completed,7.0,0.0,16.0,1.0,0.0
1,Executing,26.0,1.0,1.0,17.0,0.0
2,Released,104.0,1.0,15.0,25.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21
0,Completed,7.0,0.0,16.0,1.0,0.0
1,Executing,26.0,1.0,1.0,17.0,0.0
2,Released,104.0,1.0,15.0,25.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()